<a href="https://colab.research.google.com/github/ayalazech/MAT-421/blob/main/Module_E_Section_3_2%2C_3_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**3.2: Continunity and Differentiation**

---



In [43]:
# import library
import sympy as sym


# pythonic math expressions: add spaces, use single quotes and lowercases
# declaring variable
x = sym.symbols('x')

#limit of f as x approaches 2 
f = x**4
print(sym.limit(f, x, 2))

#limit of h as x approaches infinity 
h = ((x + 3) / (x + 2)) ** x
print(sym.limit(h, x, sym.oo)) #resulta in e

#limit of g as x approaches infinity 
g = sym.sin(x)
print(sym.limit(g, x, sym.oo))  #oscillates between -1 and 1



16
E
AccumBounds(-1, 1)


In [42]:
# declaring variables
x = sym.symbols('x')

# we take the derivative using diff
# diff (function, independent variable, number of derivatives)

exp1 = sym.diff(sym.sin(2*x), x) 
print('1st derivative: ', exp1) 

exp2 = sym.diff(sym.sin(2*x), x, 2) 
print('2nd derivative: ', exp2) 

exp3 = sym.diff(sym.sin(2*x), x, 3)
print('3rd derivative: ', exp3) 


1st derivative:  2*cos(2*x)
2nd derivative:  -4*sin(2*x)
3rd derivative:  -8*cos(2*x)


In [36]:
x, y, z = sym.symbols('x, y, z')
f = sym.exp(x * y * z)
sym.diff(f, x)  #first partial derivative with respects to x, f_(x)

y*z*exp(x*y*z)

In [38]:
sym.diff(f, x, y) #second partial derivative with respects to y, f_(xy)

z*(x*y*z + 1)*exp(x*y*z)

In [39]:
sym.diff(f, x, y, z) #third partial derivative with respects to z, f_(xyz)

(x**2*y**2*z**2 + 3*x*y*z + 1)*exp(x*y*z)


**3.3: Unconstrained Optimization**

---




In unconstrained optimization, the objective function may be a single-variable or multi-variable function, and the goal is to find the value(s) of the variable(s) that minimize or maximize the objective function. This is achieved by iteratively updating the values of the variable(s) in the direction that leads to a decrease (or increase) in the value of the objective function. The algorithm used to solve the optimization problem depends on the specific problem, but many methods such as gradient descent, Newton's method, and conjugate gradient method can be used.

Examples: Optimize the Rosenbrock function to find its minimums. 

In [12]:
#Nelder-Mead Simplex algorithm
import numpy as np
from scipy.optimize import minimize

def rosen(x):
    """The Rosenbrock function"""
    return sum(100.0*(x[1:]-x[:-1]**2.0)**2.0 + (1-x[:-1])**2.0)

x0 = np.array([1.3, 0.7, 0.8, 1.9, 1.2])
res = minimize(rosen, x0, method='nelder-mead',
               options={'xatol': 1e-8, 'disp': True})

print(res.x)

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 339
         Function evaluations: 571
[1. 1. 1. 1. 1.]


The simplex algorithm is perhaps the most straightforward method for minimizing a well-behaved function. It solely relies on function evaluations and is suitable for uncomplicated minimization tasks. Nevertheless, since it doesn't employ any gradient evaluations, it may take more time to locate the minimum.

In [13]:
#Broyden-Fletcher-Goldfarb-Shanno algorithm
def rosen_der(x):
    xm = x[1:-1]
    xm_m1 = x[:-2]
    xm_p1 = x[2:]
    der = np.zeros_like(x)
    der[1:-1] = 200*(xm-xm_m1**2) - 400*(xm_p1 - xm**2)*xm - 2*(1-xm)
    der[0] = -400*x[0]*(x[1]-x[0]**2) - 2*(1-x[0])
    der[-1] = 200*(x[-1]-x[-2]**2)
    return der

res = minimize(rosen, x0, method='BFGS', jac=rosen_der,
               options={'disp': True})

print(res.x)

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 25
         Function evaluations: 30
         Gradient evaluations: 30
[1.00000004 1.0000001  1.00000021 1.00000044 1.00000092]


To expedite convergence towards the solution, this procedure utilizes the objective function's gradient. In the event that the user doesn't provide the gradient, it's computed using first-differences. When the gradient needs to be estimated, the Broyden-Fletcher-Goldfarb-Shanno (BFGS) technique typically necessitates fewer function calls than the simplex algorithm.

In [15]:
#Newton-Conjugate-Gradient algorithm
def rosen_hess(x):
    x = np.asarray(x)
    H = np.diag(-400*x[:-1],1) - np.diag(400*x[:-1],-1)
    diagonal = np.zeros_like(x)
    diagonal[0] = 1200*x[0]**2-400*x[1]+2
    diagonal[-1] = 200
    diagonal[1:-1] = 202 + 1200*x[1:-1]**2 - 400*x[2:]
    H = H + np.diag(diagonal)
    return H

res = minimize(rosen, x0, method='Newton-CG',
               jac=rosen_der, hess=rosen_hess,
               options={'xtol': 1e-8, 'disp': True})

print(res.x)

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 24
         Function evaluations: 33
         Gradient evaluations: 33
         Hessian evaluations: 24
[1.         1.         1.         0.99999999 0.99999999]


The Newton-Conjugate Gradient algorithm modifies Newton's method and leverages a conjugate gradient algorithm to approximate the local Hessian matrix [NW]. Newton's method involves locally approximating the function with a quadratic form.

sorce:
https://docs.scipy.org/doc/scipy/tutorial/optimize.html#